In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn
import numpy as np
import time

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
start = time.time()
path = '/home/manikya_varshney/Documents/Python/Yale/Final Data/final(Cleaned and Analysed_via csv)_August18th.csv'
data = pd.read_csv(path, index_col=None, header=0, engine='python')

#path = '/home/manikya_varshney/Documents/Python/Yale/Final Data/final(Cleaned and Analysed via gzip)_August18th.gzip'
#data = pd.read_parquet(path)

data_Imp = data.copy()
data_Ada = data.copy()
data_Ne = data.copy()
data_Sd = data.copy()
data_Purp = data.copy()
data_Pe = data.copy()
end = time.time()
print("Read parquet with pandas: ",(end-start),"sec")

Read parquet with pandas:  1.935513973236084 sec


# 1_Implementation

In [3]:
keywords_Imp = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

In [4]:
#Convert to lower
for i in range(len(keywords_Imp)): 
    keywords_Imp[i] = keywords_Imp[i].lower()

#Remove punctuations   
for i in range(len(keywords_Imp)): 
    keywords_Imp[i] = keywords_Imp[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Imp)): 
    keywords_Imp[i] = keywords_Imp[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Imp_tokens = [sub.split() for sub in keywords_Imp] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Imp_filtered=remove_stopwords(keywords_Imp)

#Stemming
ps = PorterStemmer()
keywords_Imp_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Imp_filtered]
keywords_Imp_stem = [" ".join(sentence) for sentence in keywords_Imp_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Imp_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Imp_filtered]
keywords_Imp_final = [" ".join(sentence) for sentence in keywords_Imp_lem]

In [5]:
data_Imp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [6]:
data_Imp['final'].isna().value_counts()

False    227362
True        603
Name: final, dtype: int64

In [7]:
 def fuzzy_Imp(row):
        keyword_match_Imp, score_Imp = process.extractOne(row['final'], keywords_Imp_final, scorer = fuzz.partial_ratio)
        row['final_score_Imp'] = score_Imp
        row['final_keyword_match_Imp'] = keyword_match_Imp
        return row

In [8]:
nan_value = float("NaN")
data_Imp['final'].replace("", nan_value, inplace=True)

In [9]:
data_Imp.dropna(subset = ["final"], inplace=True)

In [10]:
data_Imp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [11]:
interim_Imp = data_Imp.apply(fuzzy_Imp, axis=1)

In [12]:
interim_Imp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Imp,final_keyword_match_Imp
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school,60,6feet
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...,60,6feet
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...,75,home
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...,57,closure
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...,58,stay at home
...,...,...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...,60,6feet
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...,73,responsible
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside,67,do your part
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...,50,stay at home


In [13]:
denominator_Imp = interim_Imp.shape[0]

In [14]:
interim_Imp['final_score_Imp'] = interim_Imp['final_score_Imp'].astype(int)

In [15]:
interim_Imp = interim_Imp[interim_Imp['final_score_Imp'] == 100].reset_index(drop=True)

In [16]:
interim_Imp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Imp,final_keyword_match_Imp
0,1.295805e+18,2.232332e+07,NaN,Nairobi/Kyiv/Nassau/Kosovo,"Hey, friends. . . I know a lot of y'all #homes...",hey friend know lot yall homeschool supportive...,100,home
1,1.295805e+18,2.492172e+08,NaN,NaN,#TodayInLatinAmerica: Thousands of Argentines ...,todayinlatinamerica thousand argentine protest...,100,lockdown
2,1.295805e+18,1.466334e+07,NaN,New York,"Antonio Argibay, AIA, managing principal of Me...",antonio argibay aia manage principal meridian ...,100,home
3,1.295805e+18,7.136334e+07,NaN,NaN,#VIU prof. Dr. Aggie Weighill needs your input...,viu prof dr aggie weighill need your input can...,100,distance
4,1.295805e+18,1.210700e+18,NaN,NaN,Why the Lockdown Should Last Longer https://t....,the lockdown last longer via hilarious,100,lockdown
...,...,...,...,...,...,...,...,...
36228,1.294613e+18,1.243655e+18,NaN,rent free in leftist brains,Me jamming during the height of the insane loc...,jamming the height the insane lockdown band st...,100,lockdown
36229,1.295197e+18,3.304533e+09,NaN,NaN,"@Minumuss lol, \n#covidiots \nyou test negativ...",lol covidiots test negative start isolation lm...,100,isolat
36230,1.294704e+18,1.681304e+07,NaN,Los Angeles,San Diego County has the fewest positive Covid...,san diego county the few positive covid patien...,100,lockdown
36231,1.294549e+18,1.697333e+07,NaN,"London, England",Thousands in pre-dawn scramble to beat France ...,thousand predawn scramble to beat france quara...,100,quarantine


In [17]:
numerator_Imp = interim_Imp.shape[0]

In [18]:
proportion_Imp = (numerator_Imp/denominator_Imp)
print(proportion_Imp)

0.15936260236978914


# 2_Adaptation

In [19]:
keywords_Ada = ['school from home' , 'learn', 'remote', 'school food service', 
            'online shopping', 'online purchase', 'online church', 'delivery',
            'drive thru', 'to go', 'take out', 'Tiktok', 'Netflix', 'telework', 
            'zoom', 'telehealth', 'telemedicine', 'work from home', 'wfh',
            'working at home', 'working remotely', 'online meeting']

In [20]:
#Convert to lower
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].lower()

#Remove punctuations   
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Ada_tokens = [sub.split() for sub in keywords_Ada] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Ada_filtered=remove_stopwords(keywords_Ada)

#Stemming
ps = PorterStemmer()
keywords_Ada_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Ada_filtered]
keywords_Ada_stem = [" ".join(sentence) for sentence in keywords_Ada_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Ada_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Ada_filtered]
keywords_Ada_final = [" ".join(sentence) for sentence in keywords_Ada_lem]

In [21]:
data_Ada

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [22]:
 def fuzzy_Ada(row):
        keyword_match_Ada, score_Ada = process.extractOne(row['final'], keywords_Ada_final, scorer = fuzz.partial_ratio)
        row['final_score_Ada'] = score_Ada
        row['final_keyword_match_Ada'] = keyword_match_Ada
        return row

In [23]:
nan_value = float("NaN")
data_Ada['final'].replace("", nan_value, inplace=True)

In [24]:
data_Ada.dropna(subset = ["final"], inplace=True)

In [25]:
data_Ada

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [26]:
interim_Ada = data_Ada.apply(fuzzy_Ada, axis=1)

In [27]:
denominator_Ada = interim_Ada.shape[0]

In [28]:
interim_Ada['final_score_Ada'] = interim_Ada['final_score_Ada'].astype(int)

In [29]:
interim_Ada = interim_Ada[interim_Ada['final_score_Ada'] == 100].reset_index(drop=True)

In [30]:
interim_Ada

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Ada,final_keyword_match_Ada
0,1.295805e+18,1.466334e+07,NaN,New York,"Antonio Argibay, AIA, managing principal of Me...",antonio argibay aia manage principal meridian ...,100,wfh
1,1.295805e+18,2.994861e+09,NaN,NaN,@PJS412 @Erinmc33 Patrick knows where I work. ...,patrick know work early pandemic work from hom...,100,work from home
2,1.295805e+18,9.889688e+08,NaN,NaN,@JohnWil91460279 The crazies are protesting t...,the crazy protest to vote mail say fear to go...,100,to go
3,1.295805e+18,2.913471e+09,NaN,"Omaha, Nebraska",The worst part about this remote learning is t...,the bad part remote learn strip to underwear ...,100,learn
4,1.295805e+18,4.125046e+08,NaN,"Avon, CT",An injection of the #flu vaccine will not give...,injection the flu vaccine give the flu learn,100,learn
...,...,...,...,...,...,...,...,...
8246,1.295746e+18,2.503115e+08,NaN,"Swindon, UK",Disabled workers can now get a grant worth up ...,disabled worker get grant worth to £60000 to h...,100,work from home
8247,1.295749e+18,1.280159e+18,NaN,ny4diplomaprivilege@gmail.com,Over 8.5k views of the Zoom panel! \n\n@bradho...,85k view the zoom panel bole coa need to addre...,100,zoom
8248,1.295692e+18,3.663241e+07,NaN,Nottingham,The OIA is looking for students to take part i...,the oia look student to take part next round s...,100,zoom
8249,1.294043e+18,1.411425e+07,NaN,New Orleans,The first thing the people will do once the pa...,the first thing the people do the pandemic bur...,100,zoom


In [31]:
numerator_Ada = interim_Ada.shape[0]

In [32]:
proportion_Ada = (numerator_Ada/denominator_Ada)
print(proportion_Ada)

0.03629014523095328


# 3_Negative Emotion

In [33]:
keywords_Ne = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

In [34]:
#Convert to lower
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].lower()

#Remove punctuations   
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Ne_tokens = [sub.split() for sub in keywords_Ne] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Ne_filtered=remove_stopwords(keywords_Ne)

#Stemming
ps = PorterStemmer()
keywords_Ne_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Ne_filtered]
keywords_Ne_stem = [" ".join(sentence) for sentence in keywords_Ne_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Ne_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Ne_filtered]
keywords_Ne_final = [" ".join(sentence) for sentence in keywords_Ne_lem]

In [35]:
data_Ne

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [36]:
 def fuzzy_Ne(row):
        keyword_match_Ne, score_Ne = process.extractOne(row['final'], keywords_Ne_final, scorer = fuzz.partial_ratio)
        row['final_score_Ne'] = score_Ne
        row['final_keyword_match_Ne'] = keyword_match_Ne
        return row

In [37]:
nan_value = float("NaN")
data_Ne['final'].replace("", nan_value, inplace=True)

In [38]:
data_Ne.dropna(subset = ["final"], inplace=True)

In [39]:
data_Ne

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [40]:
interim_Ne = data_Ne.apply(fuzzy_Ne, axis=1)

In [41]:
denominator_Ne = interim_Ne.shape[0]

In [42]:
interim_Ne['final_score_Ne'] = interim_Ne['final_score_Ne'].astype(int)

In [43]:
interim_Ne = interim_Ne[interim_Ne['final_score_Ne'] == 100].reset_index(drop=True)

In [44]:
interim_Ne

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Ne,final_keyword_match_Ne
0,1.295805e+18,2.232332e+07,NaN,Nairobi/Kyiv/Nassau/Kosovo,"Hey, friends. . . I know a lot of y'all #homes...",hey friend know lot yall homeschool supportive...,100,end
1,1.295805e+18,7.136334e+07,NaN,NaN,#VIU prof. Dr. Aggie Weighill needs your input...,viu prof dr aggie weighill need your input can...,100,stress
2,1.295805e+18,7.219786e+08,NaN,NaN,Why is ASUU trending? \n\nAre they still on st...,asuu trend still strike theyre also demand cov...,100,end
3,1.295805e+18,1.644156e+07,NaN,"Oklahoma, USA",Quote of the Day from State Superintendent Joy...,quote the day from state superintendent joy ho...,100,end
4,1.295805e+18,1.569142e+08,NaN,"Carmel, IN","No matter what your political views are, as ha...",no matter your political view hard may set asi...,100,end
...,...,...,...,...,...,...,...,...
11525,1.295746e+18,5.429630e+08,NaN,NaN,"At Least 30 Shot, 7 Dead, over Weekend in Demo...",at least 30 shot 7 dead weekend democratcontro...,100,end
11526,1.295650e+18,1.526502e+08,NaN,Great White North,@catonclass @schmoopgirl @Sflecce That’s becau...,sabotage create chaos crisis to push privati...,100,end
11527,1.255177e+18,6.782762e+06,NaN,United States,I’ve spent the past 48 hours—one man versus Bi...,spent the past 48 hour one man versus bill ga...,100,scar
11528,1.293872e+18,1.535155e+08,NaN,"Ottawa, Canada",Canada is moving ahead with plans to spend $1 ...,canada move ahead plan to spend 1 billion to 5...,100,end


In [45]:
numerator_Ne = interim_Ne.shape[0]

In [46]:
proportion_Ne = (numerator_Ne/denominator_Ne)
print(proportion_Ne)

0.05071208029486018


# 4_Social Disruption

In [47]:
keywords_Sd = ['social functions' , 'gathering', 'empty streets', 
            'interaction', 'large', 'no cars', 'non-essential',
            'travel', 'unnecessary', 'crowd',]

In [48]:
#Convert to lower
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].lower()

#Remove punctuations   
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Sd_tokens = [sub.split() for sub in keywords_Sd] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Sd_filtered=remove_stopwords(keywords_Sd)

#Stemming
ps = PorterStemmer()
keywords_Sd_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Sd_filtered]
keywords_Sd_stem = [" ".join(sentence) for sentence in keywords_Sd_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Sd_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Sd_filtered]
keywords_Sd_final = [" ".join(sentence) for sentence in keywords_Sd_lem]

In [49]:
data_Sd

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [50]:
 def fuzzy_Sd(row):
        keyword_match_Sd, score_Sd = process.extractOne(row['final'], keywords_Sd_final, scorer = fuzz.partial_ratio)
        row['final_score_Sd'] = score_Sd
        row['final_keyword_match_Sd'] = keyword_match_Sd
        return row

In [51]:
nan_value = float("NaN")
data_Sd['final'].replace("", nan_value, inplace=True)

In [52]:
data_Sd.dropna(subset = ["final"], inplace=True)

In [53]:
data_Sd

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [54]:
interim_Sd = data_Sd.apply(fuzzy_Sd, axis=1)

In [55]:
denominator_Sd = interim_Sd.shape[0]

In [56]:
interim_Sd['final_score_Sd'] = interim_Sd['final_score_Sd'].astype(int)

In [57]:
interim_Sd = interim_Sd[interim_Sd['final_score_Sd'] == 100].reset_index(drop=True)

In [58]:
interim_Sd

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Sd,final_keyword_match_Sd
0,1.295805e+18,2.231906e+09,NaN,Istanbul,"Norway’s sovereign wealth fund, the world’s la...",norway sovereign wealth fund the world large...,100,large
1,1.295805e+18,1.269774e+18,NaN,Oxford,@KarlTurnerMP Social distancing rules have bee...,social distance rule largely ignore since earl...,100,large
2,1.295805e+18,9.822593e+17,NaN,"Pennsylvania, USA",KOMMUNISTS KONTROL of FOOD AND TRAVEL Peggy ...,kommunists kontrol food travel peggy hall via ...,100,travel
3,1.295805e+18,1.295469e+18,NaN,NaN,Guys today i wrote my upcatet examination and ...,guy today write upcatet examination let tell o...,100,crowd
4,1.295805e+18,8.159179e+07,NaN,"New York, NY",https://t.co/M6WudONqY1 @UMassAmherst are offe...,offering wifibased contact trace app to school...,100,interaction
...,...,...,...,...,...,...,...,...
3954,1.295830e+18,1.036802e+08,NaN,Earth 🌎 🚫No lists,"Undercover Epicenter Nurse: How Fraud, Neglige...",undercover epicenter nurse fraud negligence gr...,100,unnecessary
3955,1.295513e+18,1.801077e+08,NaN,"Washington, D.C.",President Trump says he will be issuing a pard...,president trump say issue pardon tomorrow some...,100,travel
3956,1.295308e+18,1.244763e+08,NaN,Imprint:,Our Flying Reporter Bastian took the Corona qu...,fly reporter bastian take the corona quick tes...,100,travel
3957,1.295789e+18,3.848331e+07,NaN,"Boston, MA",The world's largest oil firms wiped $87bn off ...,the world large oil firm wipe 87bn the value f...,100,large


In [59]:
numerator_Sd = interim_Sd.shape[0]

In [60]:
proportion_Sd = (numerator_Sd/denominator_Sd)
print(proportion_Sd)

0.017412760267766822


# 5_Purpose

In [61]:
keywords_Purp = ['Flatten the curve' , 'Slow the spread', 'slow transmission', 
            'protect', 'save', '#stayhomesavelives']

In [62]:
#Convert to lower
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].lower()

#Remove punctuations   
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Purp_tokens = [sub.split() for sub in keywords_Purp] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Purp_filtered=remove_stopwords(keywords_Purp)

#Stemming
ps = PorterStemmer()
keywords_Purp_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Purp_filtered]
keywords_Purp_stem = [" ".join(sentence) for sentence in keywords_Purp_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Purp_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Purp_filtered]
keywords_Purp_final = [" ".join(sentence) for sentence in keywords_Purp_lem]

In [63]:
data_Purp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [64]:
 def fuzzy_Purp(row):
        keyword_match_Purp, score_Purp = process.extractOne(row['final'], keywords_Purp_final, scorer = fuzz.partial_ratio)
        row['final_score_Purp'] = score_Purp
        row['final_keyword_match_Purp'] = keyword_match_Purp
        return row

In [65]:
nan_value = float("NaN")
data_Purp['final'].replace("", nan_value, inplace=True)

In [66]:
data_Purp.dropna(subset = ["final"], inplace=True)

In [67]:
data_Purp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [68]:
interim_Purp = data_Purp.apply(fuzzy_Purp, axis=1)

In [69]:
denominator_Purp = interim_Purp.shape[0]

In [70]:
interim_Purp['final_score_Purp'] = interim_Purp['final_score_Purp'].astype(int)

In [71]:
interim_Purp = interim_Purp[interim_Purp['final_score_Purp'] == 100].reset_index(drop=True)

In [72]:
interim_Purp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Purp,final_keyword_match_Purp
0,1.295805e+18,1.212085e+18,NaN,NaN,#Checkout this #new #facemask #fineartamerica ...,checkout new facemask fineartamerica covid19 p...,100,protect
1,1.295805e+18,1.001892e+18,NaN,NaN,@marcdaalder I think we carve our own destiny....,think carve destiny build quarantine facility ...,100,save
2,1.295805e+18,2.396240e+07,NaN,Straight on 'til morning.,@TheBev_Seattle @BobFergusonAG I noticed the w...,notice the winery pandemic id love to visit th...,100,save
3,1.295805e+18,8.699340e+08,NaN,"Chariton, IA","Our Surgery Team (Dr. Rauen, Jennie Hall, and ...",surgery team dr rauen jennie hall konrad konsi...,100,protect
4,1.295805e+18,1.425606e+09,NaN,"North Carolina, USA",Following the Three Ws is a great way to #Stop...,follow the three w great way to stopthespread ...,100,protect
...,...,...,...,...,...,...,...,...
4651,1.295767e+18,1.224255e+08,NaN,"Washington, D.C.","""If Senator @RonWyden truly wants to protect s...",senator truly want to protect small business f...,100,protect
4652,1.295756e+18,2.634919e+07,NaN,NaN,So the Democrats &amp; Fauci worked with China...,the democrat amp fauci work china virus pandem...,100,protect
4653,1.280803e+18,7.421430e+05,NaN,"London, UK","""The glove or mask that you take off and casua...",the glove mask take casually disregard could e...,100,protect
4654,1.287649e+18,3.131145e+09,NaN,United Kingdom,Losing weight is hard but with some small chan...,lose weight hard small change feel fitter heal...,100,protect


In [73]:
numerator_Purp = interim_Purp.shape[0]

In [74]:
proportion_Purp = (numerator_Purp/denominator_Purp)
print(proportion_Purp)

0.020478356101723244


# 6_Positive Emotion

In [75]:
keywords_Pe = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

In [76]:
#Convert to lower
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].lower()

#Remove punctuations   
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Pe_tokens = [sub.split() for sub in keywords_Pe] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Pe_filtered=remove_stopwords(keywords_Pe)

#Stemming
ps = PorterStemmer()
keywords_Pe_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Pe_filtered]
keywords_Pe_stem = [" ".join(sentence) for sentence in keywords_Pe_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Pe_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Pe_filtered]
keywords_Pe_final = [" ".join(sentence) for sentence in keywords_Pe_lem]

In [77]:
data_Pe

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [78]:
 def fuzzy_Pe(row):
        keyword_match_Pe, score_Pe = process.extractOne(row['final'], keywords_Pe_final, scorer = fuzz.partial_ratio)
        row['final_score_Pe'] = score_Pe
        row['final_keyword_match_Pe'] = keyword_match_Pe
        return row

In [132]:
nan_value = float("NaN")
data_Pe['final'].replace("", nan_value, inplace=True)

In [133]:
data_Pe.dropna(subset = ["final"], inplace=True)

In [134]:
data_Pe

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
227960,1.295392e+18,5.676725e+07,NaN,"Chicago, IL",Bears coach Matt Nagy wore a mask the entire p...,bear coach matt nagy wore mask the entire prac...
227961,1.288855e+18,1.618015e+08,NaN,"Seattle, WA",The world’s #COVID19 response is based on inco...,the world covid19 response base incomplete se...
227962,1.294002e+18,1.197653e+18,NaN,NaN,RT if you're NOT wearing a mask outside!🙄,rt youre wear mask outside
227963,1.295370e+18,2.559617e+07,NaN,"San Diego, CA",San Diego County eligible to exit state monito...,san diego county eligible to exit state monito...


In [82]:
interim_Pe = data_Pe.apply(fuzzy_Pe, axis=1)

In [83]:
denominator_Pe = interim_Pe.shape[0]

In [84]:
interim_Pe['final_score_Pe'] = interim_Pe['final_score_Pe'].astype(int)

In [85]:
interim_Pe = interim_Pe[interim_Pe['final_score_Pe'] == 100].reset_index(drop=True)

In [86]:
interim_Pe

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Pe,final_keyword_match_Pe
0,1.295805e+18,1.170410e+18,NaN,Paris,Thank you also @GradientDragon for what turned...,thank also turn out to lovely heartfelt aftern...,100,hope
1,1.295805e+18,1.268273e+18,NaN,NaN,Our resident biology major @danie1mk says ther...,resident biology major say no hope fall footba...,100,hope
2,1.295805e+18,1.212085e+18,NaN,NaN,#Checkout this #new #facemask #fineartamerica ...,checkout new facemask fineartamerica covid19 p...,100,safe
3,1.295805e+18,1.967207e+07,NaN,"Dallas, Texas",We were so excited to host an event over the w...,excite to host event the weekend complete soci...,100,safe
4,1.295805e+18,2.693147e+09,NaN,NaN,Keeping Your Crew Safe During COVID-19...https...,keep your crew safe covid19,100,safe
...,...,...,...,...,...,...,...,...
7183,1.294361e+18,2.878549e+07,NaN,New York City / Worldwide,NEW: People who have been infected with COVID-...,new people infect covid19 recover protect to t...,100,safe
7184,1.295463e+18,8.727061e+07,NaN,"St. Paul and Minneapolis, MN","Campus is officially reopened, and we're so ex...",campus officially reopen excite to safely welc...,100,safe
7185,1.295512e+18,1.085522e+18,NaN,NaN,BTS J-Hope has donated 100 million won to the ...,bts jhope donate 100 million to the green umbr...,100,hope
7186,1.295729e+18,8.268460e+17,NaN,"New York, NY",Housing is a human right and a public health i...,housing human right public health issue everyo...,100,safe


In [87]:
numerator_Pe = interim_Pe.shape[0]

In [88]:
proportion_Pe = (numerator_Pe/denominator_Pe)
print(proportion_Pe)

0.031614781713742844


# All Proportions

In [89]:
print(proportion_Imp)

0.15936260236978914


In [90]:
print(proportion_Ada)

0.03629014523095328


In [91]:
print(proportion_Ne)

0.05071208029486018


In [92]:
print(proportion_Sd)

0.017412760267766822


In [93]:
print(proportion_Purp)

0.020478356101723244


In [94]:
print(proportion_Pe)

0.031614781713742844


# Checking for same Tweet in different Facet (15 Cases)

### 1_Implementation and 2_Adaptation

In [95]:
Imp_Ada = pd.merge(interim_Imp, interim_Ada, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [96]:
Imp_Ada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3189 entries, 0 to 3188
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    3189 non-null   float64
 1   user_id_x                             3189 non-null   float64
 2   coordinates_x                         10 non-null     object 
 3   location_x                            2439 non-null   object 
 4   extended_tweet_full_text_duplicate_x  3189 non-null   object 
 5   final_x                               3189 non-null   object 
 6   final_score_Imp                       3189 non-null   int64  
 7   final_keyword_match_Imp               3189 non-null   object 
 8   user_id_y                             3189 non-null   float64
 9   coordinates_y                         10 non-null     object 
 10  location_y                            2439 non-null   object 
 11  extended_tweet_fu

### 1_Implementation and 3_Negative Emotion

In [97]:
Imp_Ne = pd.merge(interim_Imp, interim_Ne, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [98]:
Imp_Ne.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2514 entries, 0 to 2513
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    2514 non-null   float64
 1   user_id_x                             2514 non-null   float64
 2   coordinates_x                         7 non-null      object 
 3   location_x                            1826 non-null   object 
 4   extended_tweet_full_text_duplicate_x  2514 non-null   object 
 5   final_x                               2514 non-null   object 
 6   final_score_Imp                       2514 non-null   int64  
 7   final_keyword_match_Imp               2514 non-null   object 
 8   user_id_y                             2514 non-null   float64
 9   coordinates_y                         7 non-null      object 
 10  location_y                            1826 non-null   object 
 11  extended_tweet_fu

### 1_Implementation and 4_Social Disruption

In [99]:
Imp_Sd = pd.merge(interim_Imp, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [100]:
Imp_Sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 1240
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    1241 non-null   float64
 1   user_id_x                             1241 non-null   float64
 2   coordinates_x                         3 non-null      object 
 3   location_x                            950 non-null    object 
 4   extended_tweet_full_text_duplicate_x  1241 non-null   object 
 5   final_x                               1241 non-null   object 
 6   final_score_Imp                       1241 non-null   int64  
 7   final_keyword_match_Imp               1241 non-null   object 
 8   user_id_y                             1241 non-null   float64
 9   coordinates_y                         3 non-null      object 
 10  location_y                            950 non-null    object 
 11  extended_tweet_fu

### 1_Implementation and 5_Purpose

In [101]:
Imp_Purp = pd.merge(interim_Imp, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [102]:
Imp_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 751 entries, 0 to 750
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    751 non-null    float64
 1   user_id_x                             751 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            517 non-null    object 
 4   extended_tweet_full_text_duplicate_x  751 non-null    object 
 5   final_x                               751 non-null    object 
 6   final_score_Imp                       751 non-null    int64  
 7   final_keyword_match_Imp               751 non-null    object 
 8   user_id_y                             751 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            517 non-null    object 
 11  extended_tweet_full

### 1_Implementation and 6_Positive Emotion

In [103]:
Imp_Pe = pd.merge(interim_Imp, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [104]:
Imp_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1343 entries, 0 to 1342
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    1343 non-null   float64
 1   user_id_x                             1343 non-null   float64
 2   coordinates_x                         7 non-null      object 
 3   location_x                            988 non-null    object 
 4   extended_tweet_full_text_duplicate_x  1343 non-null   object 
 5   final_x                               1343 non-null   object 
 6   final_score_Imp                       1343 non-null   int64  
 7   final_keyword_match_Imp               1343 non-null   object 
 8   user_id_y                             1343 non-null   float64
 9   coordinates_y                         7 non-null      object 
 10  location_y                            988 non-null    object 
 11  extended_tweet_fu

### 2_Adaptation and 3_Negative Emotion

In [105]:
Ada_Ne = pd.merge(interim_Ada, interim_Ne, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [106]:
Ada_Ne.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539 entries, 0 to 538
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    539 non-null    float64
 1   user_id_x                             539 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            427 non-null    object 
 4   extended_tweet_full_text_duplicate_x  539 non-null    object 
 5   final_x                               539 non-null    object 
 6   final_score_Ada                       539 non-null    int64  
 7   final_keyword_match_Ada               539 non-null    object 
 8   user_id_y                             539 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            427 non-null    object 
 11  extended_tweet_full

### 2_Adaptation and 4_Social Disruption

In [107]:
Ada_Sd = pd.merge(interim_Ada, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [108]:
Ada_Sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 191
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    192 non-null    float64
 1   user_id_x                             192 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            150 non-null    object 
 4   extended_tweet_full_text_duplicate_x  192 non-null    object 
 5   final_x                               192 non-null    object 
 6   final_score_Ada                       192 non-null    int64  
 7   final_keyword_match_Ada               192 non-null    object 
 8   user_id_y                             192 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            150 non-null    object 
 11  extended_tweet_full

### 2_Adaptation and 5_Purpose

In [109]:
Ada_Purp = pd.merge(interim_Ada, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [110]:
Ada_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 174
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    175 non-null    float64
 1   user_id_x                             175 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            134 non-null    object 
 4   extended_tweet_full_text_duplicate_x  175 non-null    object 
 5   final_x                               175 non-null    object 
 6   final_score_Ada                       175 non-null    int64  
 7   final_keyword_match_Ada               175 non-null    object 
 8   user_id_y                             175 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            134 non-null    object 
 11  extended_tweet_full

### 2_Adaptation and 6_Positive Emotion

In [111]:
Ada_Pe = pd.merge(interim_Ada, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [112]:
Ada_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334 entries, 0 to 333
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    334 non-null    float64
 1   user_id_x                             334 non-null    float64
 2   coordinates_x                         2 non-null      object 
 3   location_x                            258 non-null    object 
 4   extended_tweet_full_text_duplicate_x  334 non-null    object 
 5   final_x                               334 non-null    object 
 6   final_score_Ada                       334 non-null    int64  
 7   final_keyword_match_Ada               334 non-null    object 
 8   user_id_y                             334 non-null    float64
 9   coordinates_y                         2 non-null      object 
 10  location_y                            258 non-null    object 
 11  extended_tweet_full

### 3_Negative Emotion and 4_Social Disruption

In [113]:
Ne_Sd = pd.merge(interim_Ne, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [114]:
Ne_Sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 0 to 272
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    273 non-null    float64
 1   user_id_x                             273 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            202 non-null    object 
 4   extended_tweet_full_text_duplicate_x  273 non-null    object 
 5   final_x                               273 non-null    object 
 6   final_score_Ne                        273 non-null    int64  
 7   final_keyword_match_Ne                273 non-null    object 
 8   user_id_y                             273 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            202 non-null    object 
 11  extended_tweet_full

### 3_Negative Emotion and 5_Purpose

In [115]:
Ne_Purp = pd.merge(interim_Ne, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [116]:
Ne_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    217 non-null    float64
 1   user_id_x                             217 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            148 non-null    object 
 4   extended_tweet_full_text_duplicate_x  217 non-null    object 
 5   final_x                               217 non-null    object 
 6   final_score_Ne                        217 non-null    int64  
 7   final_keyword_match_Ne                217 non-null    object 
 8   user_id_y                             217 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            148 non-null    object 
 11  extended_tweet_full

### 3_Negative Emotion and 6_Positive Emotion

In [117]:
Ne_Pe = pd.merge(interim_Ne, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [118]:
Ne_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468 entries, 0 to 467
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    468 non-null    float64
 1   user_id_x                             468 non-null    float64
 2   coordinates_x                         1 non-null      object 
 3   location_x                            348 non-null    object 
 4   extended_tweet_full_text_duplicate_x  468 non-null    object 
 5   final_x                               468 non-null    object 
 6   final_score_Ne                        468 non-null    int64  
 7   final_keyword_match_Ne                468 non-null    object 
 8   user_id_y                             468 non-null    float64
 9   coordinates_y                         1 non-null      object 
 10  location_y                            348 non-null    object 
 11  extended_tweet_full

### 4_Social Disruption and 5_Purpose

In [119]:
Sd_Purp = pd.merge(interim_Sd, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [120]:
Sd_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 105
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    106 non-null    float64
 1   user_id_x                             106 non-null    float64
 2   coordinates_x                         1 non-null      object 
 3   location_x                            88 non-null     object 
 4   extended_tweet_full_text_duplicate_x  106 non-null    object 
 5   final_x                               106 non-null    object 
 6   final_score_Sd                        106 non-null    int64  
 7   final_keyword_match_Sd                106 non-null    object 
 8   user_id_y                             106 non-null    float64
 9   coordinates_y                         1 non-null      object 
 10  location_y                            88 non-null     object 
 11  extended_tweet_full

### 4_Social Disruption and 6_Positive Emotion

In [121]:
Sd_Pe = pd.merge(interim_Sd, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [122]:
Sd_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    187 non-null    float64
 1   user_id_x                             187 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            142 non-null    object 
 4   extended_tweet_full_text_duplicate_x  187 non-null    object 
 5   final_x                               187 non-null    object 
 6   final_score_Sd                        187 non-null    int64  
 7   final_keyword_match_Sd                187 non-null    object 
 8   user_id_y                             187 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            142 non-null    object 
 11  extended_tweet_full

### 5_Purpose and 6_Positive Emotion

In [123]:
Purp_Pe = pd.merge(interim_Purp, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [124]:
Purp_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 246
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    247 non-null    float64
 1   user_id_x                             247 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            200 non-null    object 
 4   extended_tweet_full_text_duplicate_x  247 non-null    object 
 5   final_x                               247 non-null    object 
 6   final_score_Purp                      247 non-null    int64  
 7   final_keyword_match_Purp              247 non-null    object 
 8   user_id_y                             247 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            200 non-null    object 
 11  extended_tweet_full